In [ ]:
# Import der benötigten Bibliotheken
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# Daten einlesen
gboxdf = pd.read_csv('gboxdata092025.csv', sep=';')
weatherdf = pd.read_csv('openmeteodata.csv', sep=',', skiprows=3)

# Konvertierung der Zeitstempel in datetime-Objekte
gboxdf['Timestamp'] = pd.to_datetime(gboxdf['Timestamp'])
weatherdf["date"] = pd.to_datetime(weatherdf["date"])

In [ ]:
# Korrekte Spalte für die Werte
value_column = 'Value'

# Für jeden Sensor einen DataFrame erstellen mit korrekter Filterung
sensor_101_df = gboxdf[(gboxdf['SensorID'] == 101) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', value_column]]
sensor_102_df = gboxdf[(gboxdf['SensorID'] == 102) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', value_column]]
sensor_103_df = gboxdf[(gboxdf['SensorID'] == 103) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', value_column]]
sensor_104_df = gboxdf[(gboxdf['SensorID'] == 104) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', value_column]]
sensor_105_df = gboxdf[(gboxdf['SensorID'] == 105) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', value_column]]

# Temperaturdaten filtern
temp_df = gboxdf[(gboxdf['SensorID'] == 101) & (gboxdf['MeasureName'] == 'temperature')][['Timestamp', value_column]]

In [1]:
# Umbenennung der Value-Spalte für jeden Sensor
sensor_101_df = sensor_101_df.rename(columns={value_column: 'Sensorwert101'})
sensor_102_df = sensor_102_df.rename(columns={value_column: 'Sensorwert102'})
sensor_103_df = sensor_103_df.rename(columns={value_column: 'Sensorwert103'})
sensor_104_df = sensor_104_df.rename(columns={value_column: 'Sensorwert104'})
sensor_105_df = sensor_105_df.rename(columns={value_column: 'Sensorwert105'})

NameError: name 'sensor_101_df' is not defined

In [ ]:
# Zusammenführen der DataFrames über den Timestamp
# Zuerst 101 und 102 zusammenführen
combined_df = pd.merge(sensor_101_df, sensor_102_df, on='Timestamp', how='outer')

# Dann die restlichen nacheinander hinzufügen
combined_df = pd.merge(combined_df, sensor_103_df, on='Timestamp', how='outer')
combined_df = pd.merge(combined_df, sensor_104_df, on='Timestamp', how='outer')
combined_df = pd.merge(combined_df, sensor_105_df, on='Timestamp', how='outer')

# Sortieren nach Timestamp
combined_df = combined_df.sort_values('Timestamp')

In [ ]:
# Melting der Daten für einfachere Darstellung mit Plotly
melted_df = pd.melt(combined_df,
                    id_vars=['Timestamp'],
                    value_vars=['Sensorwert101', 'Sensorwert102', 'Sensorwert103',
                               'Sensorwert104', 'Sensorwert105'],
                    var_name='Sensor', value_name='Feuchtigkeit')

In [ ]:
# Aggregierte Statistiken
daily_df = combined_df.copy()
daily_df['Datum'] = daily_df['Timestamp'].dt.date
daily_stats = daily_df.groupby('Datum').agg({
    'Sensorwert101': ['mean', 'min', 'max'],
    'Sensorwert102': ['mean', 'min', 'max'],
    'Sensorwert103': ['mean', 'min', 'max'],
    'Sensorwert104': ['mean', 'min', 'max'],
    'Sensorwert105': ['mean', 'min', 'max']
})

# Wetterdaten vorbereiten
weather_daily = weatherdf.copy()
weather_daily['date'] = pd.to_datetime(weather_daily['date']).dt.date

In [ ]:
# App erstellen
app = Dash(__name__)

# Layout der App definieren
app.layout = html.Div([
    html.H1('Uniteichpegel-Sensoranalyse', style={'textAlign': 'center'}),

    html.Div([
        html.H2('Feuchtigkeitsdaten der Sensoren'),
        html.P('Wählen Sie die anzuzeigenden Sensoren:'),
        dcc.Checklist(
            id='sensor-checklist',
            options=[
                {'label': 'Sensor 101', 'value': 'Sensorwert101'},
                {'label': 'Sensor 102', 'value': 'Sensorwert102'},
                {'label': 'Sensor 103', 'value': 'Sensorwert103'},
                {'label': 'Sensor 104', 'value': 'Sensorwert104'},
                {'label': 'Sensor 105', 'value': 'Sensorwert105'}
            ],
            value=['Sensorwert101', 'Sensorwert102', 'Sensorwert103', 'Sensorwert104', 'Sensorwert105'],
            inline=True
        ),
        dcc.Graph(id='moisture-time-series'),

        html.H3('Zeitraum auswählen:'),
        dcc.DatePickerRange(
            id='date-picker-range',
            min_date_allowed=combined_df['Timestamp'].min().date(),
            max_date_allowed=combined_df['Timestamp'].max().date(),
            start_date=combined_df['Timestamp'].min().date(),
            end_date=combined_df['Timestamp'].max().date()
        ),
    ]),

    html.Div([
        html.H2('Statistische Auswertung'),
        dcc.Dropdown(
            id='stats-dropdown',
            options=[
                {'label': 'Boxplot - Feuchtigkeitsverteilung', 'value': 'boxplot'},
                {'label': 'Heatmap - Täglicher Verlauf', 'value': 'heatmap'},
                {'label': 'Balkendiagramm - Durchschnittswerte', 'value': 'bar'}
            ],
            value='boxplot'
        ),
        dcc.Graph(id='stats-graph')
    ]),

    html.Div([
        html.H2('Wetter und Feuchtigkeit'),
        html.P('Vergleich von Sensordaten mit Wetterbedingungen:'),
        dcc.Dropdown(
            id='weather-dropdown',
            options=[col for col in weatherdf.columns if col not in ['date']],
            value='precipitation' if 'precipitation' in weatherdf.columns else weatherdf.columns[1]
        ),
        dcc.Graph(id='weather-correlation')
    ]),

    html.Div([
        html.H2('Zusammenfassung der Sensordaten'),
        html.Div(id='summary-stats')
    ]),

    html.Footer([
        html.P('Uniteichpegelbox-Analyse Dashboard © 2023', style={'textAlign': 'center'})
    ])
], style={'padding': '20px', 'fontFamily': 'Arial'})

In [ ]:
# Callbacks

@app.callback(
    Output('moisture-time-series', 'figure'),
    [Input('sensor-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_moisture_graph(selected_sensors, start_date, end_date):
    # Zeitraum filtern
    filtered_df = combined_df.copy()
    filtered_df = filtered_df[(filtered_df['Timestamp'].dt.date >= pd.to_datetime(start_date).date()) &
                             (filtered_df['Timestamp'].dt.date <= pd.to_datetime(end_date).date())]

    # Erstellen des Plots
    fig = go.Figure()

    for sensor in selected_sensors:
        fig.add_trace(go.Scatter(
            x=filtered_df['Timestamp'],
            y=filtered_df[sensor],
            mode='lines+